<a href="https://colab.research.google.com/github/wilberquito/NeuralNetworksCompetition2023/blob/main/Code/ML_STUDENT_COMPETITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M.L Competition Neural Networks

In [20]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [21]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import data_table
    data_table.DataTable.max_columns = 50
    data_table.enable_dataframe_formatter()
else:
    from IPython.core.interactiveshell import InteractiveShell
    InteractiveShell.ast_node_interactivity = "all"

In [22]:
from pathlib import Path
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from pathlib import Path


pd.set_option('display.max_columns', 500)

torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [24]:
csv_path = Path('/content/drive/MyDrive/train.csv') if IN_COLAB else Path('../Data/train.csv')
df = pd.read_csv(csv_path)
df.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,493553,0.315471,-0.183690,0.664383,-1.186794,0.665098,0.946208,0.729857,0,4,1,3,1,2,4,1,5,2,0,1,-3.690715,-0.628005,-2.832295,-1.409039,3.645067,0.233039,-3.754846,-1.061733,BDBBCACIBB,20.308715,1,0,0
1,237346,-1.286392,1.780592,0.576698,-2.690658,1.321997,-0.675894,0.371070,3,0,3,3,5,3,2,0,1,6,0,1,0.664517,-2.871912,3.826628,3.087653,0.494209,3.210875,-0.666457,0.123854,ACBDCBCADA,-449.291063,1,0,0
2,37368,-0.290303,-0.485907,0.808350,-0.156288,1.083632,-1.129914,0.767396,3,1,3,2,3,4,1,1,1,0,2,6,-0.494988,-0.946303,2.333223,2.084169,-4.782668,-1.671375,2.774382,2.273130,AABBABCLAF,-86.206118,0,1,1
3,665220,1.243590,0.035112,-1.013236,0.854267,0.019192,0.597892,-2.020416,2,0,4,5,0,5,1,0,3,1,1,3,-3.066427,-2.430158,-0.185332,-0.701691,-2.769142,-6.534231,-0.557677,-0.429972,ADBBABEEBA,-30.157403,0,2,1
4,41499,0.702716,0.817044,-0.064907,-1.045483,0.718374,0.164451,-0.936620,1,2,2,2,2,5,0,3,1,1,2,4,-1.899984,1.427460,-4.992610,1.154162,-1.931443,2.325042,2.143811,-1.039599,ABBBBBCMBB,296.484562,0,2,1


## First approuch

In [25]:
X, y = df.drop(['target', 'f_27', 'id'], axis=1), df['target']

In [26]:
X.describe()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30
count,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000
mean,0.000368,0.001564,0.000306,-0.001910,-0.000820,0.000644,-0.000971,2.032086,2.057980,2.362590,2.177847,1.804038,2.842322,2.239307,1.515411,2.101359,2.096502,1.857758,2.065975,0.307220,-0.178796,-0.156877,-0.009749,-0.369114,-0.342708,0.175932,0.356640,-0.448086,0.345565,1.002373
std,0.998719,0.999402,1.000892,1.000081,1.000000,0.999999,0.999535,1.656749,1.590698,1.637473,1.645519,1.537589,1.762672,1.537712,1.359798,1.568952,1.559978,1.467507,1.565593,2.314858,2.400672,2.484487,2.450494,2.454113,2.387102,2.416753,2.476792,238.735832,0.475553,0.818851
min,-4.599856,-4.682199,-4.642676,-4.628484,-4.748501,-4.750214,-4.842919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-11.280941,-11.257917,-12.183785,-11.853530,-12.301097,-11.416189,-11.918306,-14.300577,-1229.753052,0.000000,0.000000
25%,-0.674386,-0.674814,-0.674948,-0.676899,-0.676212,-0.672953,-0.675204,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-1.236465,-1.804803,-1.819358,-1.645711,-2.019762,-1.956021,-1.440454,-1.262614,-159.456219,0.000000,0.000000
50%,0.002005,0.002531,0.001148,-0.002556,-0.001907,0.000071,-0.001630,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,0.329103,-0.189919,-0.153055,0.030148,-0.390120,-0.342316,0.160621,0.405216,-0.573352,0.000000,1.000000
75%,0.674684,0.675737,0.676736,0.671851,0.672968,0.675826,0.673790,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,1.878621,1.444271,1.506658,1.660455,1.257267,1.266450,1.794780,2.028263,158.941960,1.000000,2.000000
max,4.749301,4.815699,4.961982,4.454920,4.948983,4.971881,4.822668,15.000000,16.000000,14.000000,14.000000,13.000000,16.000000,12.000000,14.000000,14.000000,15.000000,12.000000,13.000000,11.108150,11.475325,12.029242,11.344080,12.247100,12.389844,12.529179,12.913041,1229.562577,1.000000,2.000000


In [27]:
y.value_counts()

0    415945
1    394055
Name: target, dtype: int64

In [28]:
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [29]:
class Net(nn.Module):

    def __init__(self, input_features=30):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=128),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=128),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=128),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=1),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.main(x)

# Define a custom dataset by extending the PyTorch Dataset class
class CustomDataset(Dataset):

    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels 
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        label = self.labels[index]
        if self.transform:
            item = self.transform(item)
        return item, label
    
def datasets(dataset, stratify, test_size):
    train_size = 1 - test_size

    # Stratify by label
    labels = np.array(stratify)
    positive_indices = np.where(labels == 1)[0]
    negative_indices = np.where(labels == 0)[0]

    positive_split = int(train_size * len(positive_indices))
    negative_split = int(train_size * len(negative_indices))

    positive_train_indices = positive_indices[:positive_split]
    positive_test_indices = positive_indices[positive_split:]
    negative_train_indices = negative_indices[:negative_split]
    negative_test_indices = negative_indices[negative_split:]

    train_indices = np.concatenate([positive_train_indices, negative_train_indices])
    test_indices = np.concatenate([positive_test_indices, negative_test_indices])

    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    test_dataset = torch.utils.data.Subset(dataset, test_indices)
    
    return train_dataset, test_dataset

In [30]:
# Declare the hole dataset
X_tensor, y_tensor = torch.from_numpy(X.to_numpy()), torch.from_numpy(y.to_numpy())
X_tensor, y_tensor = X_tensor.type(torch.float), y_tensor.type(torch.float)
X_tensor, y_tensor = X_tensor.to(device), y_tensor.to(device)

In [31]:
X_tensor = (X_tensor - X_tensor.min()) / (X_tensor.max() - X_tensor.min())
X_tensor.shape, y_tensor.shape

(torch.Size([810000, 30]), torch.Size([810000]))

In [32]:
dataset = CustomDataset(data=X_tensor, labels=y_tensor)
len(dataset)

810000

In [33]:
train_dataset, test_dataset = datasets(dataset, y, test_size=0.15)
len(train_dataset), len(test_dataset)

(688499, 121501)

In [34]:
# Checkpoint
def checkpoint(epoch, model, optimizer, criteria, save_as: Path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'criteria': criteria,
    }, save_as)
    

# Validate
def validation(model, device, valid_loader, criteria):
    # Settings
    model.eval()
    loss_total = 0
    accuracy_total = 0

    # Test validation data
    with torch.inference_mode():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs).squeeze(dim=1)
            loss = criteria(outputs, labels)
            loss_total += loss.item() * inputs.size(0)
            matches = torch.round(torch.sigmoid(outputs)) == labels
            accuracy_total += (torch.sum(matches.to(torch.int32))).item()
                        
    return (loss_total / len(valid_loader), accuracy_total / len(valid_loader.dataset))


# Train
def train(device, model, epochs, optimizer, criteria, train_loader, valid_loader, resume=1, save_as=Path('../Output/model.pth')):
    # Early stopping
    last_loss = 100
    patience = 3
    trigger_times = 0
    
    train_loss_track, train_accuracy_track = [], []
    test_loss_track, test_accuracy_track = [], []

    # Move model to device
    model = model.to(device)

    for epoch in range(resume, epochs + 1):
        model.train()

        for times, (inputs, labels) in enumerate(train_loader, 1):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward and backward propagation
            outputs = model(inputs).squeeze(dim=1)
            loss = criteria(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # Train loss and train accuracy
        train_current_loss, train_current_accuracy = validation(model, device, train_loader, criteria)

        # Early stopping
        current_loss, current_accuracy = validation(model, device, valid_loader, criteria)
        print('[{}/{} | Train loss: {:.8} | Train accuracy: {:.8} | Test loss: {:.8} | Test accuracy: {:.8}]'.format(epoch, epochs, train_current_loss, train_current_accuracy, current_loss, current_accuracy))
        
        train_loss_track.append(train_current_loss)
        train_accuracy_track.append(train_current_accuracy)
        test_loss_track.append(current_loss)
        test_accuracy_track.append(current_accuracy)
        
        if current_loss > last_loss:
            trigger_times += 1
            print('Trigger times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        else:
            print('Trigger times: 0')
            trigger_times = 0

        last_loss = current_loss
        
        if trigger_times == 0:
            print('New checkpoint...')
            checkpoint(epoch, model, optimizer, criteria, save_as=save_as)

    return model, (train_loss_track, train_accuracy_track), (test_loss_track, test_accuracy_track)

In [42]:
model = Net(input_features=X_tensor.shape[1])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 100
learning_rate = 0.001
batch_size = 256
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criteria = nn.BCEWithLogitsLoss()
resume = 1
v0_path = Path('../Output/NetV0.pth')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

recover = False

if v0_path.exists() and recover:
    print('Recovering old training process...')
    checkpoint = torch.load(v0_path)
    resume = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    criteria = checkpoint['criteria']
else:
    print('Ready for new training process... =)')

Ready for new training process... =)


In [43]:
wanna_train = False
if wanna_train:
    trained_model, train_stats, test_stats = train(device, model, epochs, optimizer, criteria, train_loader, test_loader, resume=resume, save_as=v0_path)
else:
    print('Not training this model anymore...')

Not training this model anymore...


In [21]:
import shutil
v0_path_trained = Path('../Trained/NetV0.pth')
if v0_path.exists():
    print('Copying last checkpoint of model {0} state to trained folder'.format('NetV0'))
    shutil.copy(v0_path, v0_path_trained)
else:
    print('Model {0} is not created yeat'.format('NetV0'))

Copying last checkpoint of model NetV0 state to trained folder


WindowsPath('../Trained/NetV0.pth')

## Second approch

### One hot encode f_27

This is not the way to go because the dataframe increases a lot...

In [9]:
X, y = df.drop(['target', 'id'], axis=1, errors='ignore'), df['target']
X.shape, y.shape

((810000, 31), (810000,))

In [10]:
f_27 = X['f_27']
f_27.nunique(), len(f_27)

(678113, 810000)

In [11]:
from sklearn.preprocessing import OneHotEncoder

X['f_27'] = X['f_27'].astype('category')
drop_enc = OneHotEncoder(drop='first')
sparse = drop_enc.fit_transform(X[['f_27']])
sparse.shape, type(sparse)

((810000, 678112), scipy.sparse._csr.csr_matrix)

Dimentionally reduction doesn't work here

In [14]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(n_components=10)
pca.fit(sparse)

print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.sum())

print(pca.singular_values_)

TruncatedSVD(n_components=10)

[1.47292388e-05 1.33993433e-05 1.17483190e-05 1.14381288e-05
 1.05022168e-05 1.01821724e-05 9.86606175e-06 9.74468205e-06
 9.52573634e-06 8.92134886e-06]
0.00011005724817312332
[3.44591513 3.27903335 3.04689248 2.99168421 2.88523113 2.82947893
 2.78254694 2.75723908 2.72105923 2.65208615]


Clusterization doesnt perform good either

In [14]:
from sklearn.cluster import KMeans

for c in range(1, 11):
    kmeans = KMeans(
        n_clusters=c,
        max_iter=300,
        n_init='auto',
        random_state=42,
    ).fit(sparse)
    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
    print(f"Number of elements asigned to each cluster: {cluster_sizes}")
print()
print(
    "True number of documents in each category according to the class labels: "
    f"{cluster_ids}"
)

Number of elements asigned to each cluster: [810000]
Number of elements asigned to each cluster: [809996      4]
Number of elements asigned to each cluster: [809994      4      2]
Number of elements asigned to each cluster: [809993      4      2      1]
Number of elements asigned to each cluster: [809990      4      2      1      3]
Number of elements asigned to each cluster: [809988      4      2      1      3      2]
Number of elements asigned to each cluster: [809984      4      2      1      3      2      4]
Number of elements asigned to each cluster: [809980      4      2      3      2      4      3      2]
Number of elements asigned to each cluster: [809979      4      2      3      2      4      3      2      1]
Number of elements asigned to each cluster: [809978      4      2      3      2      4      3      2      1      1]

True number of documents in each category according to the class labels: [0 1 2 3 4 5 6 7 8 9]


## Smart encoding f_27

In [60]:
df['f_27'] = df['f_27'].str.upper()
df['length'] = df['f_27'].str.len()
print(df['length'].min(), df['length'].max())

10 10


In [61]:
X, y = df.drop(['target', 'id'], axis=1, errors='ignore'), df['target']
X.shape, y.shape

((810000, 32), (810000,))

In [62]:
# Counts the number of times a letter appear in the code
def add_letters_count(data):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for char in letters:
        data[char] = data['f_27'].str.count(char)
    return data

# For each of the 10 position of the letter code I assign which code letter was assigned to it
def add_letter_position(data):
    for i in range(10):
        data['pos_' + str(i)] = (data['f_27'].str[i]).apply(lambda x: ord(x)) - 65
    return data

X = add_letters_count(X)
X = add_letter_position(X)
X.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,length,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,pos_0,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9
0,0.315471,-0.183690,0.664383,-1.186794,0.665098,0.946208,0.729857,0,4,1,3,1,2,4,1,5,2,0,1,-3.690715,-0.628005,-2.832295,-1.409039,3.645067,0.233039,-3.754846,-1.061733,BDBBCACIBB,20.308715,1,0,10,1,5,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,1,1,2,0,2,8,1,1
1,-1.286392,1.780592,0.576698,-2.690658,1.321997,-0.675894,0.371070,3,0,3,3,5,3,2,0,1,6,0,1,0.664517,-2.871912,3.826628,3.087653,0.494209,3.210875,-0.666457,0.123854,ACBDCBCADA,-449.291063,1,0,10,3,2,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,3,2,1,2,0,3,0
2,-0.290303,-0.485907,0.808350,-0.156288,1.083632,-1.129914,0.767396,3,1,3,2,3,4,1,1,1,0,2,6,-0.494988,-0.946303,2.333223,2.084169,-4.782668,-1.671375,2.774382,2.273130,AABBABCLAF,-86.206118,0,1,10,4,3,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,2,11,0,5
3,1.243590,0.035112,-1.013236,0.854267,0.019192,0.597892,-2.020416,2,0,4,5,0,5,1,0,3,1,1,3,-3.066427,-2.430158,-0.185332,-0.701691,-2.769142,-6.534231,-0.557677,-0.429972,ADBBABEEBA,-30.157403,0,2,10,3,4,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1,0,1,4,4,1,0
4,0.702716,0.817044,-0.064907,-1.045483,0.718374,0.164451,-0.936620,1,2,2,2,2,5,0,3,1,1,2,4,-1.899984,1.427460,-4.992610,1.154162,-1.931443,2.325042,2.143811,-1.039599,ABBBBBCMBB,296.484562,0,2,10,1,7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,12,1,1


In [63]:
# Drops unneded features
X = X.drop(['f_27', 'length'], axis=1)
X.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,pos_0,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9
0,0.315471,-0.183690,0.664383,-1.186794,0.665098,0.946208,0.729857,0,4,1,3,1,2,4,1,5,2,0,1,-3.690715,-0.628005,-2.832295,-1.409039,3.645067,0.233039,-3.754846,-1.061733,20.308715,1,0,1,5,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,1,1,2,0,2,8,1,1
1,-1.286392,1.780592,0.576698,-2.690658,1.321997,-0.675894,0.371070,3,0,3,3,5,3,2,0,1,6,0,1,0.664517,-2.871912,3.826628,3.087653,0.494209,3.210875,-0.666457,0.123854,-449.291063,1,0,3,2,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,3,2,1,2,0,3,0
2,-0.290303,-0.485907,0.808350,-0.156288,1.083632,-1.129914,0.767396,3,1,3,2,3,4,1,1,1,0,2,6,-0.494988,-0.946303,2.333223,2.084169,-4.782668,-1.671375,2.774382,2.273130,-86.206118,0,1,4,3,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,2,11,0,5
3,1.243590,0.035112,-1.013236,0.854267,0.019192,0.597892,-2.020416,2,0,4,5,0,5,1,0,3,1,1,3,-3.066427,-2.430158,-0.185332,-0.701691,-2.769142,-6.534231,-0.557677,-0.429972,-30.157403,0,2,3,4,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1,0,1,4,4,1,0
4,0.702716,0.817044,-0.064907,-1.045483,0.718374,0.164451,-0.936620,1,2,2,2,2,5,0,3,1,1,2,4,-1.899984,1.427460,-4.992610,1.154162,-1.931443,2.325042,2.143811,-1.039599,296.484562,0,2,1,7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,12,1,1


In [64]:
# Declare the hole dataset
X_tensor, y_tensor = torch.from_numpy(X.to_numpy()), torch.from_numpy(y.to_numpy())
X_tensor, y_tensor = X_tensor.type(torch.float), y_tensor.type(torch.float)
X_tensor, y_tensor = X_tensor.to(device), y_tensor.to(device)
X_tensor

tensor([[ 0.3155, -0.1837,  0.6644,  ...,  8.0000,  1.0000,  1.0000],
        [-1.2864,  1.7806,  0.5767,  ...,  0.0000,  3.0000,  0.0000],
        [-0.2903, -0.4859,  0.8084,  ..., 11.0000,  0.0000,  5.0000],
        ...,
        [-0.2353, -0.3175, -0.3885,  ..., 11.0000,  3.0000,  3.0000],
        [-0.9948, -0.2667,  1.9090,  ..., 13.0000,  1.0000,  2.0000],
        [-0.2540,  1.2699,  1.9680,  ..., 17.0000,  1.0000,  2.0000]],
       device='cuda:0')

In [72]:
X_tensor = (X_tensor - X_tensor.min()) / (X_tensor.max() - X_tensor.min())
print(X.shape, y.shape)
X_tensor

(810000, 66) (810000,)


tensor([[0.5002, 0.5000, 0.5003,  ..., 0.5033, 0.5004, 0.5004],
        [0.4995, 0.5008, 0.5003,  ..., 0.5000, 0.5013, 0.5000],
        [0.4999, 0.4998, 0.5004,  ..., 0.5045, 0.5000, 0.5021],
        ...,
        [0.4999, 0.4999, 0.4999,  ..., 0.5045, 0.5013, 0.5013],
        [0.4996, 0.4999, 0.5008,  ..., 0.5053, 0.5004, 0.5009],
        [0.4999, 0.5006, 0.5008,  ..., 0.5070, 0.5004, 0.5009]],
       device='cuda:0')

In [82]:
dataset = CustomDataset(data=X_tensor, labels=y_tensor)

train_dataset, test_dataset = datasets(dataset, y, test_size=0.15)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

model = Net(input_features=X_tensor.shape[1])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 100
learning_rate = 0.001
batch_size = 256
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criteria = nn.BCEWithLogitsLoss()
resume = 1
v1_path = Path('../Output/NetV1.pth')

recover = False

if v_path.exists() and recover:
    print('Recovering old training process...')
    checkpoint = torch.load(v1_path)
    resume = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    criteria = checkpoint['criteria']
else:
    print('Ready for new training process... =)')

Ready for new training process... =)


In [ ]:
wanna_train = True
if wanna_train:
    trained_model, train_stats, test_stats = train(device, model, epochs, optimizer, criteria, train_loader, test_loader, resume=resume, save_as=v1_path)
else:
    print('Not training this model anymore...')

[1/100 | Train loss: 177.31987 | Train accuracy: 0.51351273 | Test loss: 177.21183 | Test accuracy: 0.51351018]
Trigger times: 1


In [ ]:
import shutil
v1_path_trained = Path('../Trained/NetV1.pth')
if v0_path.exists():
    print('Copying last checkpoint of model {0} state to trained folder'.format('NetV1'))
    shutil.copy(v1_path, v1_path_trained)
else:
    print('Model {0} is not created yeat'.format('NetV1'))